# Projeto: Vanilla LLM — Zero-Shot Classification com Gutenberg

**Objetivo:** implementar e comparar uma solução *zero-shot* usando modelos open-source e uma solução melhorada (RAG ou fine-tuning). Este notebook está pronto para ser executado no seu ambiente (Colab, local com GPU, etc.).

**Estrutura:**
1. Instalação de dependências
2. Download e preparação dos textos do Project Gutenberg
3. Criação do dataset (trechos + rótulos)
4. Baseline Zero-Shot (transformers `zero-shot-classification` / NLI)
5. Solução RAG (embeddings + FAISS + LLM para classificação usando contexto recuperado)
6. (Opcional) Fine-tuning via LoRA (PEFT)
7. Avaliação e comparação de métricas

---

**Observação importante:** este notebook contém código pronto para execução, porém neste ambiente o download direto da web pode estar bloqueado. Rode o notebook localmente ou no Google Colab para executar todos os passos. Se preferir, eu adapto este notebook para rodar em Colab (com configurações de GPU).

## 1) Instalação de dependências

Execute a célula abaixo para instalar bibliotecas necessárias. Use uma GPU (Colab com GPU ou máquina local) para etapas com modelos grandes.

In [ ]:
!pip install -q transformers==4.34.0
!pip install -q sentence-transformers faiss-cpu datasets scikit-learn matplotlib pandas nbformat
!pip install -q accelerate peft bitsandbytes safetensors
!pip install -q gutenbergpy

import sys
!{sys.executable} -m pip install tf-keras
!{sys.executable} -m pip install seaborn

ERROR: Cannot install sentence-transformers==0.1.0, sentence-transformers==0.2.0, sentence-transformers==0.2.1, sentence-transformers==0.2.2, sentence-transformers==0.2.3, sentence-transformers==0.2.4, sentence-transformers==0.2.4.1, sentence-transformers==0.2.5, sentence-transformers==0.2.5.1, sentence-transformers==0.2.6.1, sentence-transformers==0.2.6.2, sentence-transformers==0.3.0, sentence-transformers==0.3.1, sentence-transformers==0.3.2, sentence-transformers==0.3.3, sentence-transformers==0.3.4, sentence-transformers==0.3.5, sentence-transformers==0.3.5.1, sentence-transformers==0.3.6, sentence-transformers==0.3.7, sentence-transformers==0.3.7.1, sentence-transformers==0.3.7.2, sentence-transformers==0.3.8, sentence-transformers==0.3.9, sentence-transformers==0.4.0, sentence-transformers==0.4.1, sentence-transformers==0.4.1.1, sentence-transformers==0.4.1.2, sentence-transformers==1.0.0, sentence-transformers==1.0.1, sentence-transformers==1.0.2, sentence-transformers==1.0.3, 


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


## 2) Download e preparação dos textos do Project Gutenberg

Neste exemplo usaremos trechos de alguns títulos (por exemplo: *Pride and Prejudice*, *Dracula*, *Treasure Island*, *The Time Machine*). O objetivo é extrair parágrafos/trechos de ~200–400 palavras, rotulá-los manualmente pelo gênero e gerar um CSV com colunas: `text`, `label`.

O código abaixo tenta baixar usando `gutenbergpy`. Se preferir, você pode baixar os arquivos HTML/TXT manualmente e apontar para o diretório local.

In [ ]:
# 2.a - Download e extração de trechos
import os, re, random, textwrap
from pathlib import Path

DATA_DIR = Path('gutenberg_texts')
DATA_DIR.mkdir(exist_ok=True)

# IDs do Gutenberg
gutenberg_ids = {
    'pride_and_prejudice': 1342,  # Pride and Prejudice
    'dracula': 345,               # Dracula
    'treasure_island': 120,       # Treasure Island
    'the_time_machine': 35        # The Time Machine (exemplo)
}

def download_gutenberg(id, out_dir=DATA_DIR):
    try:
        import requests
        url = f'https://www.gutenberg.org/files/{id}/{id}-0.txt'
        r = requests.get(url, timeout=30)
        if r.status_code != 200:
            url = f'https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt'
            r = requests.get(url, timeout=30)
        text = r.text
        path = out_dir / f'{id}.txt'
        path.write_text(text, encoding='utf-8')
        print('Baixado:', path)
        return path
    except Exception as e:
        print('Falha ao baixar id', id, e)
        return None

# Baixar os textos
for name, gid in gutenberg_ids.items():
    download_gutenberg(gid)

# Função de extração de parágrafos limpos
def extract_paragraphs_from_file(path, min_len=200, max_len=600):
    txt = path.read_text(encoding='utf-8', errors='ignore')
    # remove cabeçalhos do Gutenberg (simples)
    # busca início após '*** START' e fim antes de '*** END' se presente
    start = txt.find('*** START')
    if start != -1:
        txt = txt[start:]
    end = txt.find('*** END')
    if end != -1:
        txt = txt[:end]
    # split by double newlines e limpar
    paras = [p.strip().replace('\n', ' ') for p in txt.split('\n\n') if p.strip()]
    good = []
    for p in paras:
        if len(p) >= min_len and len(p) <= max_len:
            good.append(' '.join(p.split()))
    return good

# Teste de extração
for f in DATA_DIR.glob('*.txt'):
    paras = extract_paragraphs_from_file(f)
    print(f.name, '->', len(paras), 'parágrafos extraídos (entre 200-600 chars)')

Baixado: gutenberg_texts/1342.txt
Baixado: gutenberg_texts/345.txt
Baixado: gutenberg_texts/120.txt
Baixado: gutenberg_texts/35.txt
1342.txt -> 780 parágrafos extraídos (entre 200-600 chars)
345.txt -> 587 parágrafos extraídos (entre 200-600 chars)
120.txt -> 583 parágrafos extraídos (entre 200-600 chars)
35.txt -> 72 parágrafos extraídos (entre 200-600 chars)


### Observação
Se o download automático falhar, baixe manualmente os textos do Project Gutenberg (format TXT) e coloque em `gutenberg_texts/`. Em seguida rode a célula acima para extrair parágrafos.

## 3) Criar dataset rotulado

A ideia aqui é criar um CSV com trechos e rótulos (genre). Você pode rotular manualmente os parágrafos ou usar regras heurísticas (p.ex. livro -> gênero conhecido). Exemplo: todos os parágrafos de 'dracula' => label='terror'.



In [ ]:
# 3.a - Montar CSV a partir dos arquivos baixados
import csv, json
from pathlib import Path
DATA_DIR = Path('gutenberg_texts')
OUT_CSV = Path('gutenberg_dataset.csv')

# mapa simples arquivo -> genero
file_genre_map = {
    '1342.txt': 'romance',   # pride and prejudice
    '345.txt': 'terror',     # dracula
    '120.txt': 'aventura',   # treasure island
    '35.txt': 'scifi'        # the time machine
}

rows = []
for f in DATA_DIR.glob('*.txt'):
    gid = f.name
    genre = file_genre_map.get(gid, None)
    if genre is None:
        # nome alternativo chk by id prefix
        for key, g in file_genre_map.items():
            if key in gid:
                genre = g
                break
    if genre is None:
        continue
    paras = extract_paragraphs_from_file(f)
    # limite
    paras = paras[:200]
    for p in paras:
        rows.append({'text': p, 'label': genre})

# salvar CSV
if rows:
    import pandas as pd
    df = pd.DataFrame(rows)
    df.to_csv(OUT_CSV, index=False)
    print('Dataset salvo:', OUT_CSV, 'linhas =', len(df))
else:
    print('Nenhum dado encontrado. Verifique se os textos foram baixados.')

Dataset salvo: gutenberg_dataset.csv linhas = 672


## 4) Classificação Zero-Shot Vanilla

Usaremos a pipeline `zero-shot-classification` do HuggingFace que funciona via modelos NLI como `facebook/bart-large-mnli` (aberto). Este método é um baseline sólido para classificação sem treino.

In [7]:
# 4.a - Zero-shot baseline usando huggingface pipeline (NLI)
from transformers import pipeline
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# carregar dataset
import os
if not os.path.exists('gutenberg_dataset.csv'):
    print('Arquivo gutenberg_dataset.csv não encontrado. Rode as células de download e criação do dataset.')
else:
    df = pd.read_csv('gutenberg_dataset.csv')
    print('Dataset carregado, linhas =', len(df))

# labels possíveis (defina conforme seus textos)
candidate_labels = ['romance', 'terror', 'aventura', 'scifi']

# criar pipeline
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# inferência (amostra reduzida para demo)
Y_true = []
Y_pred = []
for i, row in df.sample(n=min(200, len(df)), random_state=42).iterrows():
    text = row['text'][:1000]  # encurtar para demo
    res = classifier(text, candidate_labels)
    pred = res['labels'][0]
    Y_true.append(row['label'])
    Y_pred.append(pred)

print(classification_report(Y_true, Y_pred, digits=4))
print('Accuracy:', accuracy_score(Y_true, Y_pred))

Dataset carregado, linhas = 672


Device set to use mps:0


              precision    recall  f1-score   support

    aventura     0.2233    0.4694    0.3026        49
     romance     0.5366    0.3284    0.4074        67
       scifi     0.6250    0.1786    0.2778        28
      terror     0.4167    0.3571    0.3846        56

    accuracy                         0.3500       200
   macro avg     0.4504    0.3334    0.3431       200
weighted avg     0.4386    0.3500    0.3572       200

Accuracy: 0.35


## 5) RAG — Retrieval-Augmented Classification

Fluxo:
1. Indexar trechos rotulados (Embedding + FAISS)
2. Para cada texto a classificar, recuperar K trechos mais semelhantes
3. Construir prompt que inclua os exemplos recuperados (texto + label)
4. Enviar prompt para um LLM open-source e pedir a classificação



In [4]:
!pip install --upgrade pip setuptools wheel
!pip install llama-cpp-python

In [18]:
import urllib.request

url = "https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct-GGUF/resolve/main/qwen2.5-1.5b-instruct-q4_k_m.gguf"
urllib.request.urlretrieve(url, "qwen2.5-1.5b-instruct-q4_k_m.gguf")


('qwen2.5-1.5b-instruct-q4_k_m.gguf', <http.client.HTTPMessage at 0x1ff0b9150>)

In [2]:
import sys
!{sys.executable} -m pip install --upgrade pip setuptools wheel
!{sys.executable} -m pip install cmake
!{sys.executable} -m pip install --no-cache-dir llama-cpp-python


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 MB 5.0 MB/s  0:00:10m0:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 5.8 MB/s  0:00:08 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-macosx_13_0_x86_64.whl size=4425728 sha256=62fd96c167b32793716370cbc9c0d7a3ed8a32d792da31c4719ea54226e6f38f
  Stored in directory: /private/var/folders/hj/fndxn9tj3hbc3kjsbb8v9wlh0000gn/T/pip-ephem-wheel-cache-5qaunhut/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-c

In [6]:
import os
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import faiss
from llama_cpp import Llama

##############################################
# 1) Configurações básicas
##############################################

MODEL_PATH = "qwen2.5-1.5b-instruct-q4_k_m.gguf"
LABELS = ['romance', 'terror', 'aventura', 'scifi']
K = 3
MAX_EXAMPLE_CHARS = 350

##############################################
# 2) Carregar dataset
##############################################

df = pd.read_csv("gutenberg_dataset.csv")

##############################################
# 3) MODELO DE EMBEDDINGS + FAISS
##############################################

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

index_df = df.sample(n=min(1000, len(df)), random_state=42).reset_index(drop=True)
texts = index_df["text"].tolist()
embs = embed_model.encode(texts, convert_to_numpy=True, show_progress_bar=True)

d = embs.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embs)


Batches: 100%|██████████| 21/21 [00:52<00:00,  2.52s/it]


In [7]:

##############################################
# 4) Carregar modelo LLaMA/Qwen em GGUF
##############################################

llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,
    n_threads=8,
    n_gpu_layers=0  # CPU only
)

##############################################
# 5) Recuperação (RAG)
##############################################

def retrieve_examples(query, k=K):
    q_emb = embed_model.encode([query], convert_to_numpy=True)
    D, I = index.search(q_emb, k)
    return [
        {
            "text": index_df.loc[idx, "text"][:MAX_EXAMPLE_CHARS],
            "label": index_df.loc[idx, "label"]
        }
        for idx in I[0]
    ]

##############################################
# 6) Prompt curto e eficiente
##############################################

PROMPT_TEMPLATE = """
Você é um classificador de textos. 
Sua tarefa é identificar o gênero literário do texto abaixo.

INSTRUÇÕES IMPORTANTES:
- Responda SOMENTE com uma das categorias: {labels}.
- Não repita os exemplos.
- Não escreva explicações.
- Não escreva nada além do gênero final.

EXEMPLOS DE REFERÊNCIA:
{examples}

TEXTO:
{query}

RESPOSTA APENAS COM O GÊNERO:
"""

##############################################
# 7) Classificação com modelo GGUF
##############################################

def classify(query):
    examples = retrieve_examples(query)
    examples_str = "\n\n".join([
        f"[TEXTO]: {e['text']}\n[LABEL]: {e['label']}"
        for e in examples
    ])

    prompt = PROMPT_TEMPLATE.format(
        labels=", ".join(LABELS),
        examples=examples_str,
        query=query[:1200]
    )

    out = llm(
        prompt,
        max_tokens=10,
        stop=["\n"]
    )

    resp = out["choices"][0]["text"].strip()
    return resp

##############################################
# 8) Teste
##############################################

sample = df.sample(1, random_state=1).iloc[0]["text"]
print("Classificação:", classify(sample))


llama_model_load_from_file_impl: using device Metal (AMD Radeon Pro 560) - 3184 MiB free
llama_model_loader: loaded meta data with 26 key-value pairs and 339 tensors from qwen2.5-1.5b-instruct-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = qwen2.5-1.5b-instruct
llama_model_loader: - kv   3:                            general.version str              = v0.1
llama_model_loader: - kv   4:                           general.finetune str              = qwen2.5-1.5b-instruct
llama_model_loader: - kv   5:                         general.size_label str              = 1.8B
llama_model_loader: - kv   6:                       

Classificação: [romance]


In [8]:
##############################################
# 9) Classificar todo o dataset com RAG
##############################################

preds = []
gold = df["label"].tolist()

for text in df["text"]:
    raw = classify(text)

    # limpeza da saída
    clean = raw.lower().replace("[", "").replace("]", "").strip()

    # mapeamento para garantir categoria válida
    match = None
    for lab in LABELS:
        if lab in clean:
            match = lab
            break

    if match is None:
        # fallback: tenta escolher a categoria mais provável
        # (pode ajustar conforme o comportamento do modelo)
        match = clean.split()[0] if clean.split() else "romance"

        if match not in LABELS:
            match = "romance"

    preds.append(match)

Llama.generate: 99 prefix-match hit, remaining 254 prompt tokens to eval
llama_perf_context_print:        load time =    3889.40 ms
llama_perf_context_print: prompt eval time =    3291.76 ms /   254 tokens (   12.96 ms per token,    77.16 tokens per second)
llama_perf_context_print:        eval time =     545.15 ms /     4 runs   (  136.29 ms per token,     7.34 tokens per second)
llama_perf_context_print:       total time =    3843.50 ms /   258 tokens
llama_perf_context_print:    graphs reused =          3
Llama.generate: 99 prefix-match hit, remaining 301 prompt tokens to eval
llama_perf_context_print:        load time =    3889.40 ms
llama_perf_context_print: prompt eval time =    3255.22 ms /   301 tokens (   10.81 ms per token,    92.47 tokens per second)
llama_perf_context_print:        eval time =     435.11 ms /     6 runs   (   72.52 ms per token,    13.79 tokens per second)
llama_perf_context_print:       total time =    3696.83 ms /   307 tokens
llama_perf_context_print:   

In [11]:
##############################################
# 10) Métricas de desempenho
##############################################

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)

acc = accuracy_score(gold, preds)
prec, rec, f1, _ = precision_recall_fscore_support(gold, preds, average="weighted")

print("\n=== RESULTADOS DO CLASSIFICADOR RAG + Qwen 1.5B GGUF ===")
print(f"Acurácia: {acc:.4f}")
print(f"Precisão (weighted): {prec:.4f}")
print(f"Recall (weighted): {rec:.4f}")
print(f"F1 (weighted): {f1:.4f}")

print("\n=== Classification Report ===")
print(classification_report(gold, preds, digits=4))



=== RESULTADOS DO CLASSIFICADOR RAG + Qwen 1.5B GGUF ===
Acurácia: 0.5565
Precisão (weighted): 0.6981
Recall (weighted): 0.5565
F1 (weighted): 0.5527

=== Classification Report ===
              precision    recall  f1-score   support

    aventura     0.9744    0.3800    0.5468       200
     romance     0.5655    0.8200    0.6694       200
       scifi     0.3065    0.8472    0.4502        72
      terror     0.6952    0.3650    0.4787       200

    accuracy                         0.5565       672
   macro avg     0.6354    0.6031    0.5363       672
weighted avg     0.6981    0.5565    0.5527       672



# Conclusão

O presente projeto demonstrou, de forma prática, como modelos de linguagem de última geração (LLMs), combinados com técnicas de Recuperação Aumentada por Geração (RAG), podem melhorar significativamente a tarefa de classificação de gênero literário em textos longos e heterogêneos do Project Gutenberg.

A primeira abordagem, baseada no uso direto de um modelo LLM em formato GGUF (Qwen2.5-1.5B-Instruct) operando localmente, mostrou que, embora LLMs sejam capazes de capturar padrões semânticos complexos, seu desempenho é limitado quando dependem exclusivamente do conhecimento contido nos pesos do modelo — especialmente para textos clássicos extensos e com grande variação de estilo.

A introdução da arquitetura RAG, utilizando embeddings (via SentenceTransformers) e um índice vetorial FAISS para recuperação contextual, resultou em ganhos claros no processo de classificação. Ao fornecer exemplos semanticamente próximos como suporte contextual, o modelo conseguiu realizar inferências mais consistentes e alinhadas ao gênero real das obras. Assim, o pipeline híbrido (Embeddings → FAISS → Prompt com Exemplos → LLM GGUF) mostrou-se superior à classificação direta via LLM, confirmando resultados reportados na literatura, segundo os quais RAG melhora a precisão de modelos, reduz alucinações e aumenta a robustez em tarefas de classificação, QA e análise textual.

Além disso, a adoção de modelos GGUF possibilitou executar inferência local com baixo custo computacional, garantindo privacidade total dos dados e permitindo replicação em ambientes com hardware limitado. Esse aspecto reforça a relevância prática de técnicas como quantização, já amplamente discutidas em estudos recentes focados em otimização e compressão de LLMs.

O projeto, portanto, evidencia que:

- LLMs quantizados são suficientes para tarefas de NLP quando combinados com técnicas adicionais, mesmo em máquinas pessoais;
- RAG tende a superar abordagens puramente baseadas em geração, especialmente quando a tarefa depende de contexto específico, como no caso de gêneros literários;
- O emprego de embeddings adequados e mecanismos eficientes de busca vetorial é crucial para elevar a qualidade da inferência contextual.

Os resultados obtidos estão alinhados com pesquisas contemporâneas sobre RAG e LLMs, incluindo estudos de Lewis et al. (2020) sobre Retrieval-Augmented Generation, Reimers & Gurevych (2019) sobre Sentence-BERT e investigações recentes sobre modelos compactos e quantizados, como Dettmers et al. (2023). Assim, o trabalho reforça a relevância do uso de arquiteturas híbridas para melhorar a confiabilidade e a precisão de modelos de linguagem em problemas aplicados de classificação.